# ⚡ High Performance Python

Bienvenue dans ce module où tu vas apprendre à **accélérer considérablement** tes pipelines Python. Tu découvriras comment contourner les limitations du GIL, paralléliser tes traitements, et gérer des fichiers plus grands que ta RAM !

---

## 📋 Prérequis

| Niveau | Compétence |
|--------|------------|
| ✅ Requis | Maîtriser Python (fonctions, classes) |
| ✅ Requis | Avoir suivi le module `17_polars` |
| 💡 Recommandé | Connaître Pandas |

## 🎯 Objectifs du module

À la fin de ce module, tu seras capable de :

- Comprendre le **GIL** et ses implications sur la performance
- **Profiler** ton code pour identifier les vrais goulots d'étranglement
- Utiliser **concurrent.futures** pour paralléliser simplement
- Maîtriser **asyncio** pour l'I/O massivement parallèle
- Exploiter **Dask** pour traiter des fichiers plus grands que la RAM
- Choisir le **bon outil** selon ton problème

<a href="https://colab.research.google.com/github/diakite-data/data-engineering-bootcamp/blob/main/notebooks/intermediate/18_high_performance_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>> 💡 **Conseil** : Cliquez sur le badge ci-dessus pour exécuter ce notebook directement dans Google Colab (aucune installation requise).

---

## 🔒 1. Le GIL : Comprendre la limitation fondamentale

> 🧠 **Cette section est essentielle** pour comprendre pourquoi certaines techniques fonctionnent et d'autres non.

### 1.1 Qu'est-ce que le GIL ?

Le **Global Interpreter Lock (GIL)** est un verrou qui empêche Python d'exécuter plusieurs threads Python **simultanément**.

```text
AVEC LE GIL (Python standard)
═════════════════════════════

Thread 1: ████████░░░░░░░░████████░░░░░░░░
Thread 2: ░░░░░░░░████████░░░░░░░░████████
          ──────────────────────────────▶ temps
          
→ Les threads s'exécutent en alternance, pas en parallèle !


SANS GIL (multiprocessing)
══════════════════════════

Process 1: ████████████████████████████████
Process 2: ████████████████████████████████
           ──────────────────────────────▶ temps
           
→ Vraie exécution parallèle sur plusieurs CPUs !
```

### 1.2 CPU-bound vs I/O-bound

| Type | Exemple | GIL bloquant ? | Solution |
|------|---------|----------------|----------|
| **CPU-bound** | Calculs, transformations, ETL | ✅ **Oui** | `multiprocessing`, `ProcessPoolExecutor` |
| **I/O-bound** | API, fichiers, base de données | ❌ Non | `threading`, `asyncio` |

**Pourquoi le GIL ne bloque pas l'I/O ?**

Quand Python attend une réponse réseau ou disque, il **relâche le GIL**. Un autre thread peut donc s'exécuter pendant ce temps d'attente.

> ℹ️ **Le savais-tu ?**
>
> Le GIL a été introduit dans Python pour **simplifier la gestion de la mémoire**. Il rend Python thread-safe par défaut, mais au prix de la performance multi-thread.
>
> Des projets comme **nogil** (Python 3.13+) et **subinterpreters** travaillent à supprimer ou contourner cette limitation.
>
> En attendant, les Data Engineers utilisent **multiprocessing** pour contourner le GIL !

In [ ]:
import time
import threading
from multiprocessing import Process

def cpu_intensive(n):
    """Tâche CPU-bound : calcul intensif"""
    total = 0
    for i in range(n):
        total += i ** 2
    return total

N = 5_000_000

# ============ SÉQUENTIEL ============
start = time.time()
cpu_intensive(N)
cpu_intensive(N)
seq_time = time.time() - start
print(f"⏱️ Séquentiel : {seq_time:.2f}s")

# ============ THREADING (bloqué par GIL) ============
start = time.time()
t1 = threading.Thread(target=cpu_intensive, args=(N,))
t2 = threading.Thread(target=cpu_intensive, args=(N,))
t1.start(); t2.start()
t1.join(); t2.join()
thread_time = time.time() - start
print(f"🧵 Threading : {thread_time:.2f}s (GIL bloque!)")

# ============ MULTIPROCESSING (contourne GIL) ============
start = time.time()
p1 = Process(target=cpu_intensive, args=(N,))
p2 = Process(target=cpu_intensive, args=(N,))
p1.start(); p2.start()
p1.join(); p2.join()
proc_time = time.time() - start
print(f"🚀 Multiprocessing : {proc_time:.2f}s (vraie parallélisation!)")

print(f"\n📊 Speedup multiprocessing vs séquentiel : {seq_time/proc_time:.1f}x")

---

## 🔍 2. Identifier les goulots : Profiling

> ⚠️ **"Premature optimization is the root of all evil"** — Donald Knuth
>
> Avant d'optimiser, il faut **mesurer** pour identifier le vrai problème.

### 2.1 Outils de profiling

| Outil | Usage | Comment l'utiliser |
|-------|-------|--------------------|
| `%%time` | Temps d'une cellule | Jupyter magic |
| `%%timeit` | Temps moyen (plusieurs runs) | Jupyter magic |
| `cProfile` | Profiling par fonction | `python -m cProfile script.py` |
| `line_profiler` | Profiling ligne par ligne | `@profile` decorator |
| `memory_profiler` | Usage RAM | `@profile` + `mprof run` |

In [ ]:
# %%time - mesure le temps d'exécution d'une cellule
import time

def slow_function():
    total = 0
    for i in range(1_000_000):
        total += i
    return total

%time result = slow_function()

In [ ]:
# %%timeit - moyenne sur plusieurs exécutions
def fast_function():
    return sum(range(1_000_000))

%timeit fast_function()

In [ ]:
import cProfile
import pstats
from io import StringIO

def main_pipeline():
    """Pipeline simulé avec plusieurs étapes"""
    data = list(range(100_000))
    
    # Étape 1 : transformation
    transformed = [x ** 2 for x in data]
    
    # Étape 2 : filtrage
    filtered = [x for x in transformed if x % 2 == 0]
    
    # Étape 3 : agrégation
    result = sum(filtered)
    
    return result

# Profiler le code
profiler = cProfile.Profile()
profiler.enable()

result = main_pipeline()

profiler.disable()

# Afficher les résultats
stream = StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')
stats.print_stats(10)
print(stream.getvalue())

---

## 🗺️ 3. Stratégies de performance : Vue d'ensemble

| Besoin | Solution | Quand l'utiliser |
|--------|----------|------------------|
| Multi-CPU (CPU-bound) | `ProcessPoolExecutor` | ETL lourd, calculs |
| I/O parallèle (simple) | `ThreadPoolExecutor` | < 20 requêtes/fichiers |
| I/O parallèle (massif) | `asyncio` | 100+ requêtes API |
| Gros fichiers (> RAM) | Polars streaming, Dask | 10-100+ Go |
| Parallélisation simple | `joblib` | Boucles, ML |

### 🌳 Arbre de décision

```text
Ton problème est...
│
├─▶ CPU-bound (calculs, transformations) ?
│   ├─▶ Simple/boucle → joblib
│   └─▶ Complexe/chunks → ProcessPoolExecutor
│
├─▶ I/O-bound (API, fichiers, DB) ?
│   ├─▶ < 20 requêtes → ThreadPoolExecutor
│   └─▶ 100+ requêtes → asyncio
│
└─▶ Gros fichiers (> RAM) ?
    ├─▶ Single file, Polars-like → Polars streaming
    ├─▶ Multi-files, Pandas-like → Dask
    └─▶ Cluster distribué → Spark (module 19)
```

---

## 🎯 4. concurrent.futures — L'API moderne

> ✅ **Plus simple** que `multiprocessing` et `threading` bruts
> ✅ **Interface unifiée** pour threads et processes

### 4.1 ThreadPoolExecutor (I/O-bound)

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import time

def simulate_io_task(task_id):
    """Simule une tâche I/O (ex: requête API)"""
    time.sleep(0.5)  # Simule latence réseau
    return f"Task {task_id} completed"

tasks = list(range(10))

# ============ SÉQUENTIEL ============
start = time.time()
results_seq = [simulate_io_task(t) for t in tasks]
print(f"⏱️ Séquentiel : {time.time() - start:.2f}s")

# ============ THREADPOOL ============
start = time.time()
with ThreadPoolExecutor(max_workers=5) as executor:
    results_parallel = list(executor.map(simulate_io_task, tasks))
print(f"🚀 ThreadPool : {time.time() - start:.2f}s")

print(f"\n📊 Speedup : {5.0 / (time.time() - start + 0.01):.1f}x environ")

### 4.2 ProcessPoolExecutor (CPU-bound)

In [ ]:
from concurrent.futures import ProcessPoolExecutor
import time
import os

def cpu_task(n):
    """Tâche CPU-intensive"""
    return sum(i ** 2 for i in range(n))

# Données à traiter
data_chunks = [1_000_000] * 8  # 8 chunks

# ============ SÉQUENTIEL ============
start = time.time()
results_seq = [cpu_task(chunk) for chunk in data_chunks]
seq_time = time.time() - start
print(f"⏱️ Séquentiel : {seq_time:.2f}s")

# ============ PROCESSPOOL ============
start = time.time()
with ProcessPoolExecutor(max_workers=4) as executor:
    results_parallel = list(executor.map(cpu_task, data_chunks))
proc_time = time.time() - start
print(f"🚀 ProcessPool : {proc_time:.2f}s")

print(f"\n📊 Speedup : {seq_time/proc_time:.1f}x")
print(f"💻 CPUs disponibles : {os.cpu_count()}")

### 4.3 Gestion avancée : submit() et as_completed()

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import random

def fetch_data(task_id):
    """Simule une requête avec temps variable"""
    delay = random.uniform(0.1, 1.0)
    time.sleep(delay)
    return {"task_id": task_id, "delay": delay}

# Utiliser submit() pour plus de contrôle
with ThreadPoolExecutor(max_workers=5) as executor:
    # Soumettre les tâches
    futures = {executor.submit(fetch_data, i): i for i in range(5)}
    
    # Traiter les résultats au fur et à mesure qu'ils arrivent
    for future in as_completed(futures):
        task_id = futures[future]
        try:
            result = future.result(timeout=5)  # Timeout de 5s
            print(f"✅ Task {task_id} terminée en {result['delay']:.2f}s")
        except Exception as e:
            print(f"❌ Task {task_id} a échoué : {e}")

### 4.4 Quand utiliser quoi ?

| Executor | GIL contourné ? | Usage | Exemple |
|----------|-----------------|-------|----------|
| `ThreadPoolExecutor` | ❌ Non | I/O : API, fichiers | Télécharger 50 fichiers |
| `ProcessPoolExecutor` | ✅ Oui | CPU : calculs, ETL | Transformer 8 chunks de données |

---

## 🔧 5. multiprocessing — Contrôle avancé

Pour les cas où `concurrent.futures` ne suffit pas.

### 5.1 Pool avec map

In [ ]:
from multiprocessing import Pool, cpu_count
import numpy as np

def process_chunk(chunk):
    """Traite un chunk de données"""
    return np.sum(chunk ** 2)

# Créer des données
big_array = np.random.rand(1_000_000)

# Partitionner en chunks
n_workers = cpu_count()
chunks = np.array_split(big_array, n_workers)

print(f"💻 Nombre de workers : {n_workers}")
print(f"📦 Taille des chunks : {[len(c) for c in chunks]}")

# Traitement parallèle
with Pool(processes=n_workers) as pool:
    results = pool.map(process_chunk, chunks)

total = sum(results)
print(f"\n✅ Résultat total : {total:.2f}")

### 5.2 starmap pour plusieurs arguments

In [ ]:
from multiprocessing import Pool

def process_with_params(data, multiplier, offset):
    """Fonction avec plusieurs paramètres"""
    return sum(data) * multiplier + offset

# Préparer les arguments
args_list = [
    ([1, 2, 3], 2, 10),
    ([4, 5, 6], 3, 20),
    ([7, 8, 9], 4, 30),
]

with Pool(3) as pool:
    results = pool.starmap(process_with_params, args_list)

print("Résultats:", results)

### 5.3 Limites et précautions

| ⚠️ Limite | Explication |
|-----------|-------------|
| **Overhead** | Créer des process prend du temps (~100ms) |
| **Sérialisation** | Les données sont copiées (pickle) |
| **`if __name__ == "__main__"`** | Obligatoire sur Windows |
| **Mémoire** | Chaque process a sa propre mémoire |

```python
# ⚠️ Toujours protéger avec if __name__ == "__main__"
if __name__ == "__main__":
    with Pool(4) as pool:
        results = pool.map(my_func, data)
```

---

## 🌐 6. asyncio — I/O massivement parallèle

> ✅ **Idéal pour** : ingestion depuis API, requêtes DB/S3 massives
> ❌ **Pas pour** : calculs CPU-intensive

### 6.1 Concepts : async/await

In [ ]:
import asyncio
import time

async def fetch_data(task_id):
    """Simule une requête API asynchrone"""
    print(f"🚀 Début tâche {task_id}")
    await asyncio.sleep(1)  # Simule latence réseau (non-bloquant !)
    print(f"✅ Fin tâche {task_id}")
    return f"result_{task_id}"

async def main():
    # Lancer 5 tâches en parallèle
    tasks = [fetch_data(i) for i in range(5)]
    results = await asyncio.gather(*tasks)
    return results

# Exécuter
start = time.time()
results = await main()  # Dans Jupyter, pas besoin de asyncio.run()
print(f"\n⏱️ Temps total : {time.time() - start:.2f}s (au lieu de 5s séquentiel)")
print(f"📊 Résultats : {results}")

### 6.2 Exemple réel avec aiohttp

In [ ]:
# Installation : pip install aiohttp
import asyncio

# Simulons aiohttp pour l'exemple (sans vraies requêtes)
async def fetch_url(session, url):
    """Simule une requête HTTP"""
    await asyncio.sleep(0.1)  # Simule latence
    return {"url": url, "status": 200}

async def fetch_all_urls(urls):
    """Fetch toutes les URLs en parallèle"""
    session = None  # En vrai : async with aiohttp.ClientSession() as session:
    
    tasks = [fetch_url(session, url) for url in urls]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    return results

# Simuler 20 URLs
urls = [f"https://api.example.com/data/{i}" for i in range(20)]

start = time.time()
results = await fetch_all_urls(urls)
print(f"⏱️ 20 requêtes en {time.time() - start:.2f}s")
print(f"✅ Succès : {len([r for r in results if isinstance(r, dict)])}")

### 6.3 Semaphore : limiter les connexions simultanées

In [ ]:
import asyncio
import time

# Limiter à 5 connexions simultanées
semaphore = asyncio.Semaphore(5)

async def fetch_limited(task_id):
    """Fetch avec limite de concurrence"""
    async with semaphore:  # Attend si déjà 5 en cours
        print(f"🚀 Tâche {task_id} démarre")
        await asyncio.sleep(0.5)
        print(f"✅ Tâche {task_id} terminée")
        return task_id

async def main():
    tasks = [fetch_limited(i) for i in range(15)]
    return await asyncio.gather(*tasks)

start = time.time()
results = await main()
print(f"\n⏱️ 15 tâches (max 5 simultanées) en {time.time() - start:.2f}s")

### 6.4 Quand NE PAS utiliser asyncio

| Situation | asyncio efficace ? | Alternative |
|-----------|-------------------|-------------|
| 100+ appels API | ✅ **Oui** | - |
| Lecture S3/DB massives | ✅ **Oui** | - |
| Calculs CPU | ❌ **Non** | ProcessPoolExecutor |
| 5 requêtes simples | ❌ Overkill | ThreadPoolExecutor |
| Code synchrone existant | ❌ Refactoring lourd | ThreadPoolExecutor |

---

## 📊 7. Dask — Pandas distribué

> 🔥 **Le plus utile avant Spark** pour traiter des fichiers plus grands que la RAM.

### 7.1 Pourquoi Dask ?

| Avantage | Description |
|----------|-------------|
| **API Pandas-like** | Migration facile |
| **Lazy evaluation** | Optimisation automatique |
| **Parallélisme** | Utilise tous les CPUs |
| **Out-of-core** | Fichiers > RAM |
| **Scalable** | Optionnellement sur cluster |

In [ ]:
# Installation : pip install dask[complete]
import dask.dataframe as dd
import pandas as pd
import os

# Créer des fichiers de test
os.makedirs("data/dask_demo", exist_ok=True)

for i in range(5):
    df = pd.DataFrame({
        "id": range(i * 10000, (i + 1) * 10000),
        "category": [f"cat_{j % 5}" for j in range(10000)],
        "amount": [float(j % 1000) for j in range(10000)]
    })
    df.to_csv(f"data/dask_demo/file_{i}.csv", index=False)

print("✅ 5 fichiers CSV créés (50,000 lignes au total)")

In [ ]:
import dask.dataframe as dd
import time

# Lire TOUS les fichiers avec glob pattern (lazy !)
ddf = dd.read_csv("data/dask_demo/*.csv")

print("Type:", type(ddf))
print(f"Partitions: {ddf.npartitions}")
print("\n⚠️ Les données ne sont PAS encore chargées !")
print(ddf)  # Affiche le schéma, pas les données

In [ ]:
# Pipeline Dask (lazy)
result = (
    ddf
    .filter(ddf.amount > 100)  # Filtrage
    .assign(amount_doubled=ddf.amount * 2)  # Transformation
    .groupby("category")  # Agrégation
    .amount.sum()
)

print("Pipeline défini (lazy) :")
print(result)

# Exécuter avec compute()
start = time.time()
final_result = result.compute()
print(f"\n⏱️ Exécution : {time.time() - start:.2f}s")
print("\nRésultat :")
print(final_result)

### 7.2 Dask vs Polars vs Spark

| Aspect | Polars | Dask | Spark |
|--------|--------|------|-------|
| **Single machine** | ⭐⭐⭐ | ⭐⭐ | ⭐ |
| **Cluster** | ❌ | ⭐⭐ | ⭐⭐⭐ |
| **API** | Propre | Pandas-like | Propre |
| **Setup** | Simple | Simple | Complexe |
| **Vitesse (single node)** | ⭐⭐⭐ | ⭐⭐ | ⭐ |
| **Écosystème** | Nouveau | Mature | Très mature |

### 7.3 Quand utiliser Dask ?

- ✅ Fichiers **> RAM** mais **< 100 Go**
- ✅ Tu connais déjà **Pandas**
- ✅ Pas besoin d'un **cluster Spark**
- ✅ Traitement **multi-fichiers**
- ❌ Si single file < 10 Go → utilise **Polars**
- ❌ Si > 100 Go ou cluster → utilise **Spark**

---

## 🎨 8. joblib — Parallélisation simple

> ✅ **Ultra-simple** — parfait pour paralléliser une boucle rapidement
> ✅ Très utilisé en **Data Science** (sklearn l'utilise en interne)

In [ ]:
# Installation : pip install joblib
from joblib import Parallel, delayed
import time

def expensive_computation(x):
    """Calcul coûteux"""
    time.sleep(0.1)  # Simule un calcul
    return x ** 2

data = list(range(20))

# ============ SÉQUENTIEL ============
start = time.time()
results_seq = [expensive_computation(x) for x in data]
print(f"⏱️ Séquentiel : {time.time() - start:.2f}s")

# ============ JOBLIB ============
start = time.time()
results_parallel = Parallel(n_jobs=-1)(  # -1 = tous les CPUs
    delayed(expensive_computation)(x) for x in data
)
print(f"🚀 Joblib : {time.time() - start:.2f}s")

In [ ]:
from joblib import Parallel, delayed

def io_task(x):
    time.sleep(0.1)
    return x

# Options utiles
results = Parallel(
    n_jobs=4,              # Nombre de workers
    backend="threading",   # "threading" pour I/O, "loky" (défaut) pour CPU
    verbose=1              # Affiche la progression
)(delayed(io_task)(x) for x in range(10))

print(f"\n✅ Résultats : {results}")

---

## 🌳 9. Choisir la bonne technologie

### Récapitulatif

| Outil | Type | GIL contourné | Complexité | Use case |
|-------|------|---------------|------------|----------|
| `ThreadPoolExecutor` | I/O | ❌ Non | ⭐ | < 20 requêtes/fichiers |
| `ProcessPoolExecutor` | CPU | ✅ Oui | ⭐⭐ | ETL, calculs |
| `asyncio` | I/O | ❌ Non | ⭐⭐⭐ | 100+ requêtes API |
| `joblib` | CPU/I/O | ✅ (loky) | ⭐ | Boucles simples, ML |
| `Dask` | Big Data | ✅ Oui | ⭐⭐ | Fichiers > RAM |

### 🖼️ Arbre de décision visuel

```text
                    ┌─────────────────┐
                    │  Quel problème? │
                    └────────┬────────┘
                             │
         ┌───────────────────┼───────────────────┐
         │                   │                   │
         ▼                   ▼                   ▼
    ┌─────────┐        ┌─────────┐        ┌─────────────┐
    │CPU-bound│        │I/O-bound│        │Fichiers >RAM│
    └────┬────┘        └────┬────┘        └──────┬──────┘
         │                  │                    │
    ┌────┴────┐        ┌────┴────┐         ┌────┴────┐
    │         │        │         │         │         │
    ▼         ▼        ▼         ▼         ▼         ▼
 Simple?  Complex?  <20 req?  100+ req?  <100Go?  >100Go?
    │         │        │         │         │         │
    ▼         ▼        ▼         ▼         ▼         ▼
 joblib   Process   Thread    asyncio    Dask     Spark
          Pool      Pool                        (mod 19)
```

---

## ⚠️ 10. Bonnes pratiques & Erreurs fréquentes

### ❌ Erreurs fréquentes

| Erreur | Pourquoi c'est faux | Solution |
|--------|---------------------|----------|
| Threading pour CPU | GIL bloque | `ProcessPoolExecutor` |
| Async pour calculs | Pas de gain | `ProcessPoolExecutor` |
| Pandas sur 50 Go | Crash RAM | Dask ou Polars streaming |
| 100 workers pour 10 tâches | Overhead inutile | Adapter au workload |
| Pas de profiling | Optimise au hasard | Toujours profiler d'abord |
| Oublier `if __name__` | Crash sur Windows | Toujours protéger |

### ✅ Bonnes pratiques

| Pratique | Pourquoi |
|----------|----------|
| **Profiler d'abord** | Identifier le vrai goulot |
| **Écrire en Parquet** | I/O plus rapide |
| **Partitionner intelligemment** | Évite surcharge mémoire |
| **Tester avec peu de workers** | Puis augmenter progressivement |
| **`if __name__ == "__main__"`** | Obligatoire pour multiprocessing |
| **Utiliser `n_jobs=-1`** | Utilise tous les CPUs disponibles |

---

## 🧪 Quiz de fin de module

---

### ❓ Q1. Qu'est-ce que le GIL empêche en Python ?
a) L'exécution de code Python  
b) L'exécution simultanée de plusieurs threads Python  
c) L'utilisation de la mémoire  
d) La lecture de fichiers

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : b** — Le GIL (Global Interpreter Lock) empêche l'exécution simultanée de plusieurs threads Python, les forçant à s'exécuter en alternance.

</details>

---

### ❓ Q2. Pour un traitement CPU-intensive, quel outil utiliser ?
a) `ThreadPoolExecutor`  
b) `ProcessPoolExecutor`  
c) `asyncio`  
d) Tous sont équivalents

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : b** — `ProcessPoolExecutor` contourne le GIL en utilisant des processus séparés, permettant une vraie parallélisation CPU.

</details>

---

### ❓ Q3. Quand utiliser `asyncio` ?
a) Pour des calculs mathématiques complexes  
b) Pour télécharger 100+ fichiers depuis une API  
c) Pour trier un gros tableau  
d) Pour compresser des fichiers

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : b** — `asyncio` est idéal pour l'I/O massivement parallèle (requêtes API, téléchargements). Les autres sont CPU-bound.

</details>

---

### ❓ Q4. Que fait `ddf.compute()` dans Dask ?
a) Définit le pipeline  
b) Affiche le schéma  
c) Déclenche l'exécution et retourne un DataFrame Pandas  
d) Sauvegarde les données

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : c** — `.compute()` déclenche l'exécution du pipeline lazy et retourne le résultat sous forme de DataFrame Pandas.

</details>

---

### ❓ Q5. Que signifie `n_jobs=-1` dans joblib ?
a) Désactive le parallélisme  
b) Utilise un seul CPU  
c) Utilise tous les CPUs disponibles  
d) Erreur de configuration

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : c** — `n_jobs=-1` indique à joblib d'utiliser tous les CPUs disponibles sur la machine.

</details>

---

### ❓ Q6. Pourquoi `ThreadPoolExecutor` ne accélère pas les calculs CPU ?
a) Parce qu'il est mal implémenté  
b) Parce que le GIL force l'exécution séquentielle des threads Python  
c) Parce qu'il utilise trop de mémoire  
d) Parce qu'il est obsolète

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : b** — Le GIL empêche les threads Python de s'exécuter en parallèle. Pour du CPU-bound, il faut utiliser des processus.

</details>

---

### ❓ Q7. Pour traiter un fichier de 50 Go avec une API Pandas-like, quel outil choisir ?
a) Pandas  
b) Polars  
c) Dask  
d) asyncio

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : c** — Dask permet de traiter des fichiers plus grands que la RAM avec une API similaire à Pandas.

</details>

---

### ❓ Q8. Quelle est la première étape avant d'optimiser du code ?
a) Ajouter du multiprocessing  
b) Réécrire en Rust  
c) Profiler pour identifier le goulot d'étranglement  
d) Utiliser asyncio

<details><summary>💡 Voir la réponse</summary>

✅ **Réponse : c** — "Premature optimization is the root of all evil". Il faut d'abord mesurer pour savoir où optimiser.

</details>

---

## 🚀 Mini-projet : Pipeline haute performance

### 🎯 Objectif
Combiner plusieurs techniques pour créer un pipeline performant :
- **ProcessPoolExecutor** pour transformation CPU-intensive
- **Dask** pour agrégation
- Export **Parquet**

### 🏗️ Architecture

```text
data/raw/*.csv
      │
      ▼
┌─────────────────────┐
│ ProcessPoolExecutor │  Transformation parallèle (CPU)
│  - Nettoyage        │
│  - Feature eng.     │
└──────────┬──────────┘
           │
           ▼
data/intermediate/*.csv
           │
           ▼
┌─────────────────────┐
│   Dask DataFrame    │  Agrégation (multi-fichiers)
└──────────┬──────────┘
           │
           ▼
data/processed/result.parquet
```

In [ ]:
# Setup : créer les données de test
import pandas as pd
import numpy as np
import os

os.makedirs("data/raw", exist_ok=True)
os.makedirs("data/intermediate", exist_ok=True)
os.makedirs("data/processed", exist_ok=True)

# Créer 10 fichiers CSV (simule des logs)
np.random.seed(42)
categories = ["web", "api", "db", "cache", "auth"]
statuses = ["success", "error", "timeout"]

for i in range(10):
    n_rows = 10000
    df = pd.DataFrame({
        "timestamp": pd.date_range("2024-01-01", periods=n_rows, freq="s"),
        "category": np.random.choice(categories, n_rows),
        "status": np.random.choice(statuses, n_rows, p=[0.8, 0.15, 0.05]),
        "response_time_ms": np.random.exponential(100, n_rows),
        "bytes_sent": np.random.randint(100, 10000, n_rows)
    })
    df.to_csv(f"data/raw/logs_{i:02d}.csv", index=False)

print("✅ 10 fichiers CSV créés (100,000 lignes au total)")

In [ ]:
from concurrent.futures import ProcessPoolExecutor
import pandas as pd
import numpy as np
import glob
import time

def transform_file(filepath):
    """
    Transformation CPU-intensive d'un fichier.
    - Nettoyage
    - Feature engineering
    - Export intermédiaire
    """
    # Lire
    df = pd.read_csv(filepath)
    
    # Nettoyage : filtrer les timeouts extrêmes
    df = df[df["response_time_ms"] < 10000]
    
    # Feature engineering (CPU-intensive)
    df["response_time_log"] = np.log1p(df["response_time_ms"])
    df["is_error"] = (df["status"] != "success").astype(int)
    df["throughput"] = df["bytes_sent"] / (df["response_time_ms"] + 1)
    
    # Extraction date
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df["hour"] = df["timestamp"].dt.hour
    df["day_of_week"] = df["timestamp"].dt.dayofweek
    
    # Export intermédiaire
    output_path = filepath.replace("raw", "intermediate")
    df.to_csv(output_path, index=False)
    
    return output_path

# Liste des fichiers
input_files = sorted(glob.glob("data/raw/*.csv"))
print(f"📁 {len(input_files)} fichiers à traiter")

# ============ TRANSFORMATION PARALLÈLE ============
start = time.time()

with ProcessPoolExecutor(max_workers=4) as executor:
    output_files = list(executor.map(transform_file, input_files))

print(f"\n⏱️ Transformation : {time.time() - start:.2f}s")
print(f"✅ {len(output_files)} fichiers transformés")

In [ ]:
import dask.dataframe as dd
import time

# ============ AGRÉGATION AVEC DASK ============
start = time.time()

# Lire tous les fichiers intermédiaires (lazy)
ddf = dd.read_csv("data/intermediate/*.csv")

# Pipeline d'agrégation
result = (
    ddf
    .groupby(["category", "status", "hour"])
    .agg({
        "response_time_ms": ["mean", "max", "count"],
        "bytes_sent": "sum",
        "is_error": "sum",
        "throughput": "mean"
    })
    .compute()  # Exécution
)

# Aplatir les colonnes multi-index
result.columns = ['_'.join(col).strip() for col in result.columns.values]
result = result.reset_index()

print(f"⏱️ Agrégation Dask : {time.time() - start:.2f}s")
print(f"\n📊 Résultat : {len(result)} lignes")
print(result.head(10))

In [ ]:
# ============ EXPORT PARQUET ============
result.to_parquet("data/processed/aggregated_logs.parquet", index=False)
print("✅ Résultat exporté : data/processed/aggregated_logs.parquet")

# Vérification
import os
size_bytes = os.path.getsize("data/processed/aggregated_logs.parquet")
print(f"📦 Taille : {size_bytes / 1024:.1f} KB")

In [ ]:
# ============ RÉSUMÉ DU PIPELINE ============
print("\n" + "="*50)
print("📊 RÉSUMÉ DU PIPELINE HAUTE PERFORMANCE")
print("="*50)
print(f"""  
1️⃣ Input : 10 fichiers CSV (100K lignes)
2️⃣ Transformation : ProcessPoolExecutor (4 workers)
   - Nettoyage
   - Feature engineering
3️⃣ Agrégation : Dask DataFrame
   - GroupBy multi-colonnes
   - Aggregations multiples
4️⃣ Output : Parquet ({size_bytes / 1024:.1f} KB)
""")

---

## 📚 Ressources pour aller plus loin

### 🌐 Documentation officielle
- [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html) — Doc Python
- [asyncio](https://docs.python.org/3/library/asyncio.html) — Doc Python
- [Dask Documentation](https://docs.dask.org/) — Guide complet
- [joblib](https://joblib.readthedocs.io/) — Parallélisation simple

### 📖 Articles & Tutoriels
- [Real Python - Async IO](https://realpython.com/async-io-python/) — Tutoriel complet
- [Speed Up Your Python Code](https://realpython.com/python-concurrency/) — Guide concurrence

### 🔧 Outils de profiling
- [py-spy](https://github.com/benfred/py-spy) — Sampling profiler
- [Scalene](https://github.com/plasma-umass/scalene) — CPU + mémoire + GPU profiler

---

## ➡️ Prochaine étape

Maintenant que tu maîtrises les techniques de performance en Python, passons au **traitement distribué à grande échelle** avec Spark !

👉 **Module suivant : `19_pyspark_advanced.ipynb`** — PySpark Avancé

Tu vas apprendre :
- Architecture Spark (Driver, Executors)
- RDDs et DataFrames Spark
- Optimisations (partitioning, caching)
- Spark sur Kubernetes

---

### 📝 Récapitulatif de ce module

| Outil | Type | Quand l'utiliser |
|-------|------|------------------|
| `ThreadPoolExecutor` | I/O | < 20 requêtes/fichiers |
| `ProcessPoolExecutor` | CPU | Calculs, transformations |
| `asyncio` | I/O massif | 100+ requêtes API |
| `joblib` | Simple | Paralléliser une boucle |
| `Dask` | Big Data | Fichiers > RAM, API Pandas |

---

🎉 **Félicitations !** Tu as terminé le module High Performance Python.

In [ ]:
# Nettoyage des fichiers temporaires (optionnel)
import shutil
import os

# Décommenter pour nettoyer
# for folder in ["data/raw", "data/intermediate", "data/processed", "data/dask_demo"]:
#     if os.path.exists(folder):
#         shutil.rmtree(folder)
# print("🧹 Fichiers temporaires supprimés")